## Pair Programming ETL Transformación I
### Modulo 2 Sprint 2
#### Lola Rubio y Judith Mellidez 

Tendréis que usar el csv attacks_limpieza_completa que tenéis adjunto abajo.

En la lección de hoy aprendimos como transformar nuestros datos para que estén preparados para almacearlos en una BBDD. En este momento tenemos dos fuentes de datos:

In [1]:
from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all" # Nos permite mostar más de una salida por celda

import requests
import pandas as pd
import numpy as np
import ast

1. El csv con los ataques de tiburones que hemos estado limpiando hasta ahora, el que os hemos adjuntado (attacks_limpieza_completa). Sentiros libres de usar vuestros propios csv en caso de que queráis.
2. El csv con los datos climáticos de los principales paises que tienen ataques de tiburones, el que creamos en el pair programming de ayer.

El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes. Para ello:

- Cargaremos los dos ficheros de datos

In [2]:
#Cargamos el df realizado en ETLI datos climaticos.
df_climatico = pd.read_csv('attacks_climaticoETLI.csv')
df_climatico.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,lon,lat,pais
0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 3}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 210, 'speed':...",12,15,4,1026,none,0,0,270,2,-100.445882,39.78373,USA
1,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 345, 'speed':...",12,15,3,1027,none,0,0,15,2,-100.445882,39.78373,USA


In [3]:
#Cargamos el df attacks facilitado por Ana. 
df_attacks =pd.read_csv('attacks_limpieza_completa.csv',index_col = 0) 
df_attacks.tail(3)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,...,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
6215,0,Unprovoked,australia,27.0,Tiger shark,Unknown,Y,M,-24.776109,134.755000,...,0,0,1,0,0,1,2,14,7,6.313925
6216,0,Unprovoked,panama,27.0,Unespecific,Unknown,Y,M,8.559559,-81.130843,...,0,0,1,0,0,1,3,14,7,6.313925
6217,0,Unprovoked,ceylon (sri lanka),15.0,Unespecific,Unknown,Y,M,7.555494,80.713785,...,0,0,1,0,0,1,3,14,7,4.581659


In [4]:
#Aplicamos metodo 'isin' 
df_paises_climaticos =df_attacks[df_attacks['country'].isin(['usa','australia','new zealand','south africa','papua new guinea'])]
df_paises_climaticos.sample(3)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,...,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
3075,1979,Unprovoked,usa,27.0,Tiger shark,Mar,N,M,39.783730,-100.445882,...,1,0,0,1,0,0,2,8,7,6.313925
3792,1962,Sea Disaster,usa,27.0,Unespecific,May,Y,M,39.783730,-100.445882,...,0,0,1,0,0,1,3,9,6,6.313925
3313,1972,Unprovoked,australia,27.0,Unespecific,Feb,N,M,-24.776109,134.755000,...,1,0,0,1,0,0,3,4,7,6.313925


In [5]:
df_climatico.dtypes

timepoint              int64
cloudcover             int64
highcloud              int64
midcloud               int64
lowcloud               int64
rh_profile            object
wind_profile          object
temp2m                 int64
lifted_index           int64
rh2m                   int64
msl_pressure           int64
prec_type             object
prec_amount            int64
snow_depth             int64
wind10m.direction      int64
wind10m.speed          int64
lon                  float64
lat                  float64
pais                  object
dtype: object

In [6]:
df_climatico['rh_profile'] = df_climatico['rh_profile'].apply(ast.literal_eval)

In [7]:
rh = df_climatico['rh_profile'].apply(pd.Series)
rh.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'rh': 3}","{'layer': '900mb', 'rh': 0}","{'layer': '850mb', 'rh': -2}","{'layer': '800mb', 'rh': -1}","{'layer': '750mb', 'rh': 0}","{'layer': '700mb', 'rh': 2}","{'layer': '650mb', 'rh': 4}","{'layer': '600mb', 'rh': 6}","{'layer': '550mb', 'rh': 7}","{'layer': '500mb', 'rh': 7}","{'layer': '450mb', 'rh': 5}","{'layer': '400mb', 'rh': 1}","{'layer': '350mb', 'rh': -1}","{'layer': '300mb', 'rh': 2}","{'layer': '250mb', 'rh': 2}","{'layer': '200mb', 'rh': 4}"
1,"{'layer': '950mb', 'rh': 2}","{'layer': '900mb', 'rh': 0}","{'layer': '850mb', 'rh': 0}","{'layer': '800mb', 'rh': 1}","{'layer': '750mb', 'rh': 3}","{'layer': '700mb', 'rh': 5}","{'layer': '650mb', 'rh': 7}","{'layer': '600mb', 'rh': 7}","{'layer': '550mb', 'rh': 5}","{'layer': '500mb', 'rh': 3}","{'layer': '450mb', 'rh': 2}","{'layer': '400mb', 'rh': 0}","{'layer': '350mb', 'rh': 4}","{'layer': '300mb', 'rh': 5}","{'layer': '250mb', 'rh': 7}","{'layer': '200mb', 'rh': 7}"


In [8]:
#Hacemos un for loop para desempaquetar la columna rh_profile
for i in range(len(rh.columns)): 
    name = "rh_new" + str(rh[i].apply(pd.Series)['layer'][0])
    value = list(rh[i].apply(pd.Series)["rh"] )
    df_climatico.insert(i, name, value)

In [9]:
df_climatico.head(2)

,rh_new950mb,rh_new900mb,rh_new850mb,rh_new800mb,rh_new750mb,rh_new700mb,rh_new650mb,rh_new600mb,rh_new550mb,rh_new500mb,...,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,lon,lat,pais
0,3,0,-2,-1,0,2,4,6,7,7,...,4,1026,none,0,0,270,2,-100.445882,39.78373,USA
1,2,0,0,1,3,5,7,7,5,3,...,3,1027,none,0,0,15,2,-100.445882,39.78373,USA


In [10]:
df_climatico['wind_profile'] = df_climatico['wind_profile'].apply(ast.literal_eval)

In [11]:
wind = df_climatico['wind_profile'].apply(pd.Series)
wind.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'direction': 210, 'speed': 2}","{'layer': '900mb', 'direction': 250, 'speed': 2}","{'layer': '850mb', 'direction': 300, 'speed': 3}","{'layer': '800mb', 'direction': 305, 'speed': 4}","{'layer': '750mb', 'direction': 315, 'speed': 4}","{'layer': '700mb', 'direction': 320, 'speed': 5}","{'layer': '650mb', 'direction': 315, 'speed': 5}","{'layer': '600mb', 'direction': 310, 'speed': 5}","{'layer': '550mb', 'direction': 305, 'speed': 5}","{'layer': '500mb', 'direction': 305, 'speed': 6}","{'layer': '450mb', 'direction': 310, 'speed': 6}","{'layer': '400mb', 'direction': 325, 'speed': 6}","{'layer': '350mb', 'direction': 330, 'speed': 7}","{'layer': '300mb', 'direction': 335, 'speed': 7}","{'layer': '250mb', 'direction': 335, 'speed': 8}","{'layer': '200mb', 'direction': 340, 'speed': 8}"
1,"{'layer': '950mb', 'direction': 345, 'speed': 2}","{'layer': '900mb', 'direction': 35, 'speed': 2}","{'layer': '850mb', 'direction': 355, 'speed': 3}","{'layer': '800mb', 'direction': 325, 'speed': 3}","{'layer': '750mb', 'direction': 315, 'speed': 3}","{'layer': '700mb', 'direction': 310, 'speed': 4}","{'layer': '650mb', 'direction': 310, 'speed': 5}","{'layer': '600mb', 'direction': 305, 'speed': 5}","{'layer': '550mb', 'direction': 305, 'speed': 5}","{'layer': '500mb', 'direction': 310, 'speed': 5}","{'layer': '450mb', 'direction': 325, 'speed': 6}","{'layer': '400mb', 'direction': 330, 'speed': 7}","{'layer': '350mb', 'direction': 325, 'speed': 7}","{'layer': '300mb', 'direction': 325, 'speed': 7}","{'layer': '250mb', 'direction': 335, 'speed': 7}","{'layer': '200mb', 'direction': 340, 'speed': 8}"


In [12]:
#Creamos una variable donde extraemos la key 'layer' convertida en string  
for i in range(len(wind.columns)): 
    name = 'wind_direct' + str(wind[i].apply(pd.Series)['layer'][0])
    values = list(wind[i].apply(pd.Series)['direction'] )
    df_climatico.insert(i, name, values)

In [13]:
for i in range(len(wind.columns)): 
    name2 ='wind_speed' + str(wind[i].apply(pd.Series)['layer'][0])
    values2 = list(wind[i].apply(pd.Series)['speed'])
    df_climatico.insert(i, name2, values2)

In [14]:
df_climatico.head(2)

,wind_speed950mb,wind_speed900mb,wind_speed850mb,wind_speed800mb,wind_speed750mb,wind_speed700mb,wind_speed650mb,wind_speed600mb,wind_speed550mb,wind_speed500mb,...,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,lon,lat,pais
0,2,2,3,4,4,5,5,5,5,6,...,4,1026,none,0,0,270,2,-100.445882,39.78373,USA
1,2,2,3,3,3,4,5,5,5,5,...,3,1027,none,0,0,15,2,-100.445882,39.78373,USA


In [15]:
df_climatico.shape

(320, 67)

In [16]:
#Borramos las columnas que continen informacion duplicada
df_climatico.drop(['rh_profile','wind_profile'], axis=1, inplace=True)

In [17]:
df_climatico.shape

(320, 65)

In [18]:
#Sacamos las medias de las medidas climaticas por cada pais.
df_climatico=df_climatico.groupby('pais').mean()
df_climatico.head(5)

,wind_speed950mb,wind_speed900mb,wind_speed850mb,wind_speed800mb,wind_speed750mb,wind_speed700mb,wind_speed650mb,wind_speed600mb,wind_speed550mb,wind_speed500mb,...,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,lon,lat
pais,,,,,,,,,,,,,,,,,,,,,
Australia,3.671875,3.640625,3.500000,3.406250,3.375000,3.390625,3.312500,3.421875,3.703125,3.843750,...,25.890625,-3.750000,10.906250,1014.906250,2.859375,0.0,122.968750,3.296875,134.755000,-24.776109
New Zealand,3.687500,3.625000,3.500000,3.500000,3.484375,3.546875,3.609375,3.765625,3.984375,4.140625,...,15.062500,9.093750,10.859375,1015.500000,3.203125,0.0,149.453125,3.359375,172.834408,-41.500083
Papua New Guinea,3.828125,4.250000,4.296875,4.109375,3.406250,2.828125,2.437500,2.734375,3.218750,3.703125,...,25.890625,-0.671875,11.500000,1010.265625,2.109375,0.0,84.375000,3.156250,144.248908,-5.681607
South Africa,2.359375,2.265625,2.218750,2.140625,2.156250,2.234375,2.484375,2.781250,3.062500,3.375000,...,23.421875,2.562500,10.156250,1019.484375,1.781250,0.0,152.500000,2.296875,24.991639,-28.816624
USA,2.343750,2.515625,2.953125,3.296875,3.531250,3.937500,4.296875,4.484375,4.609375,4.734375,...,12.718750,13.265625,4.812500,1022.968750,0.000000,0.0,203.750000,2.093750,-100.445882,39.783730


In [19]:
#Poner todos los valores de una columna en minuscula para poder mergear ambos df
df_climatico = df_climatico.reset_index()

def lower_words (col): 
        col=col.lower()
        return col

In [20]:
df_climatico['country']=df_climatico['pais'].apply(lower_words)
df_climatico.head(2)

,pais,wind_speed950mb,wind_speed900mb,wind_speed850mb,wind_speed800mb,wind_speed750mb,wind_speed700mb,wind_speed650mb,wind_speed600mb,wind_speed550mb,...,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,lon,lat,country
0,Australia,3.671875,3.640625,3.5,3.40625,3.375000,3.390625,3.312500,3.421875,3.703125,...,-3.75000,10.906250,1014.90625,2.859375,0.0,122.968750,3.296875,134.755000,-24.776109,australia
1,New Zealand,3.687500,3.625000,3.5,3.50000,3.484375,3.546875,3.609375,3.765625,3.984375,...,9.09375,10.859375,1015.50000,3.203125,0.0,149.453125,3.359375,172.834408,-41.500083,new zealand


In [21]:
#Borramos una de las columnas una vez cambiados los nombres a minuscula.
df_climatico.drop(['pais'], axis=1, inplace=True)

In [22]:
df_climatico.head()

,wind_speed950mb,wind_speed900mb,wind_speed850mb,wind_speed800mb,wind_speed750mb,wind_speed700mb,wind_speed650mb,wind_speed600mb,wind_speed550mb,wind_speed500mb,...,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,lon,lat,country
0,3.671875,3.640625,3.500000,3.406250,3.375000,3.390625,3.312500,3.421875,3.703125,3.843750,...,-3.750000,10.906250,1014.906250,2.859375,0.0,122.968750,3.296875,134.755000,-24.776109,australia
1,3.687500,3.625000,3.500000,3.500000,3.484375,3.546875,3.609375,3.765625,3.984375,4.140625,...,9.093750,10.859375,1015.500000,3.203125,0.0,149.453125,3.359375,172.834408,-41.500083,new zealand
2,3.828125,4.250000,4.296875,4.109375,3.406250,2.828125,2.437500,2.734375,3.218750,3.703125,...,-0.671875,11.500000,1010.265625,2.109375,0.0,84.375000,3.156250,144.248908,-5.681607,papua new guinea
3,2.359375,2.265625,2.218750,2.140625,2.156250,2.234375,2.484375,2.781250,3.062500,3.375000,...,2.562500,10.156250,1019.484375,1.781250,0.0,152.500000,2.296875,24.991639,-28.816624,south africa
4,2.343750,2.515625,2.953125,3.296875,3.531250,3.937500,4.296875,4.484375,4.609375,4.734375,...,13.265625,4.812500,1022.968750,0.000000,0.0,203.750000,2.093750,-100.445882,39.783730,usa


In [28]:
df_union_clima_attacks = df_attacks.merge(df_climatico, how='inner',  on='country') 
#Unimos mediante la interseccion de las claves de ambos df.


In [29]:
df_union_clima_attacks.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,...,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,lon,lat
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,...,12.71875,13.265625,4.8125,1022.96875,0.0,0.0,203.75,2.09375,-100.445882,39.78373
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,...,12.71875,13.265625,4.8125,1022.96875,0.0,0.0,203.75,2.09375,-100.445882,39.78373


In [30]:
df_union_clima_attacks.shape

(4335, 84)

In [31]:
df_union_clima_attacks.to_csv('attacks_climaticoETLII.csv')